# Polars Tutorial

In [ ]:
import pandas as pd
import polars as pl
from pathlib import Path

# Motivation

## Separate Data from Logic
- Speed optimization
- Method chaining
- --> better code & can customize logic more extensively


## Speed
- Built-in parallelization + many other optimizations
- ~10-100X faster than pandas
<img src="assets/benchmark.png" alt="image" style="width:1200px;">

## Memory Optimization
- Larger than memory data
- More efficient storage --> can load more data into memory
- Wes McKinney's rule of thumb (pandas): "you should have 5 to 10 times as much RAM as the size of your dataset". 💀❌💀

## Syntax
- Method chaining --> one interaction with data --> better code
- Better dtypes (formal lists, structs, strict numerics)
- No more indexes!! (column nor row)

## Multi-Language
- Python
- Rust
- JS
- R (nascent)
- Ruby (nascent)

## Downsides
- Less popular (for now...)
    - ==> less stackoverflow/etc
    - ==> less LLM knowledge
- Can be verbose, especially at first

# Tutorial

## Sample Dataset

In [ ]:
DATA_PATH = Path("data/trade_i_baci_a_17__2022.parquet")

### Dataset Preview

**Shape:** (11,375,504, 12)

| year | exporter_id | exporter_name | importer_id | ... | value  | quantity | unit_abbrev | unit_name   |
|------|-------------|---------------|-------------|-----|--------|----------|-------------|-------------|
| i64  | str         | str           | str         |     | f64    | f64      | str         | str         |
| 2022 | BEL         | Belgium       | POL         | ... | 328.0  | 0.019    | mt          | Metric Tons |
| 2022 | BEL         | Belgium       | POL         | ... | 54.0   | 0.026    | mt          | Metric Tons |
| 2022 | BEL         | Belgium       | POL         | ... | 690.0  | 1.022    | mt          | Metric Tons |
| 2022 | BEL         | Belgium       | POL         | ... | 159.0  | 0.061    | mt          | Metric Tons |
| 2022 | BEL         | Belgium       | POL         | ... | 219.0  | 0.184    | mt          | Metric Tons |
| ...  | ...         | ...           | ...         | ... | ...    | ...      | ...         | ...         |
| 2022 | GHA         | Ghana         | NOR         | ... | 1579.0 | 0.105    | mt          | Metric Tons |
| 2022 | GHA         | Ghana         | NOR         | ... | 2208.0 | 0.099    | mt          | Metric Tons |
| 2022 | GHA         | Ghana         | NOR         | ... | 156.0  | 0.018    | mt          | Metric Tons |
| 2022 | GHA         | Ghana         | NOR         | ... | 38.0   | 0.001    | mt          | Metric Tons |
| 2022 | GHA         | Ghana         | NOR         | ... | 644.0  | 0.007    | mt          | Metric Tons |

## Example: read data

### Pandas

In [ ]:
pd_df: pd.DataFrame = pd.read_parquet(DATA_PATH)

#### Check Memory Usage

In [ ]:
pd_df.memory_usage(deep=True).sum() / (1024**2)  # convert bytes to MB

### Polars

In [ ]:
pl_df: pl.dataframe = pl.read_parquet(DATA_PATH)

#### Check Memory Usage

In [ ]:
pl_df.estimated_size() / (1024**2)  # convert bytes to MB

## Example: Select Data

`["exporter_id"]`  

`["exporter_id", "exporter_name"]`

In [ ]:
display(pd_df["exporter_id"])
display(pd_df[["exporter_id", "exporter_name"]])

In [ ]:
display(pl_df.select(pl.col("exporter_id")))
display(pl_df.select(pl.col(["exporter_id", "exporter_name"])))

### Select data by type

string columns

In [ ]:
pl_df.select(pl.col(pl.String))

### Select Modified Data

Example 1: select a string column and split it into a list column, on the letter "L"

Example 2: Example 1 + take the first element of the resulting arrays

In [ ]:
pd_df["exporter_id"].str.split("L")

In [ ]:
pl_df["exporter_id"].str.split("L")

In [ ]:
pl_df.select(pl.col("exporter_id").str.split("L"))
pl_df.select(pl.col("exporter_id").str.split("L").list[0])

## Example: Filter (Substring Search)

find `cocoa` in the `product_name` column (case-insensitive)

In [ ]:
pd_df[pd_df["product_name"].str.contains("(?i)cocoa", regex=True)]

In [ ]:
pl_df.filter(pl.col("product_name").str.contains("(?i)cocoa"))

## Example: Group



In [ ]:
pd_df.groupby("exporter_id").agg({"value": "sum", "quantity": ["sum", "mean"]})

In [ ]:
pl_df.group_by("exporter_id").agg(
    pl.col("value").sum(),
    pl.col("quantity").sum(),
    pl.col("quantity").mean().alias("quantity_mean"),
).sort("exporter_id")

In [ ]:
pl_df

## Example: Add a column

In [ ]:
pd_df.assign(price_per_unit=(pd_df["value"] / pd_df["quantity"]))

In [ ]:
pl_df.with_columns((pl.col("value") / pl.col("quantity")).alias("price_per_unit"))

ppu_exprs = (pl.col("value") / pl.col("quantity")).alias("price_per_unit")
print(ppu_exprs)
pl_df.with_columns(ppu_exprs)

### Complex Query

1. filter `exporter_id` contains "A|B"  
& (`product_name`) starts OR ends with "s"

2. add a `price_per_unit` column

3. group by `["year", "exporter_id", "importer_id"]`

4. sum `value`

In [ ]:
def complex_query_pd(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df[
            df["exporter_id"].str.contains("A|B")
            & (
                df["product_name"].str.endswith("s")
                | df["product_name"].str.startswith("s")
            )
        ]
        .assign(price_per_unit=(pd_df["value"] / pd_df["quantity"]))
        .groupby(["year", "exporter_id", "importer_id"])
        .agg({"value": "sum"})
    )

In [ ]:
pd_df.pipe(complex_query_pd)

In [ ]:
def complex_query_pl(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df.filter(
            (pl.col("exporter_id").str.contains("A|B"))
            & (
                pl.col("product_name").str.ends_with("s")
                | pl.col("product_name").str.starts_with("s")
            )
        )
        .with_columns(price_per_unit=(pl.col("value") / pl.col("quantity")))
        .group_by(["year", "exporter_id", "importer_id"])
        .agg(pl.col("value").sum())
    )

# def complex_query_pl_expr_vars(df: pl.DataFrame) -> pl.DataFrame:
#     exporter_contains_ab_expr: pl.Expr = ...
#     product_name_starts_ends_with_s_expr: pl.Expr = ...
#     price_per_unit_expr: pl.Expr = ...
#     agg_expr: pl.Expr = ...
#     return (
#         df.filter(exporter_contains_ab_expr & product_name_starts_ends_with_s_expr)
#         .with_columns(price_per_unit_expr)
#         .group_by(["year", "exporter_id", "importer_id"])
#         .agg(agg_expr)
#     )

In [ ]:
complex_query_pl(pl_df)

### Complex Query — Lazy!?

In [ ]:
def complex_query_pl_lf(lf: pl.LazyFrame) -> pl.LazyFrame:
    return (
        lf.filter(
            (pl.col("exporter_id").str.contains("A|B"))
            & (
                pl.col("product_name").str.ends_with("s")
                | pl.col("product_name").str.starts_with("s")
            )
        )
        .with_columns(price_per_unit=(pl.col("value") / pl.col("quantity")))
        .group_by(["year", "exporter_id", "importer_id"])
        .agg(pl.col("value").sum())
    )

In [ ]:
complex_query_output_lf = complex_query_pl_lf(pl_df.lazy())

In [ ]:
print(complex_query_output_lf)

In [ ]:
complex_query_output_lf

In [ ]:
complex_query_output_lf.show_graph()

In [ ]:
complex_query_output_lf.collect()

### Complex Query — From Disk

Repeat the complex query—including reading the data in from disk—but only `"scan"` the polars data before performing the complex query

#### pandas

In [ ]:
def complex_query_from_disk_pd() -> pd.DataFrame:
    df = pd.read_parquet(DATA_PATH)
    return (
        df[
            df["exporter_id"].str.contains("A|B")
            & (
                df["product_name"].str.endswith("s")
                | df["product_name"].str.startswith("s")
            )
        ]
        .assign(price_per_unit=(pd_df["value"] / pd_df["quantity"]))
        .groupby(["year", "exporter_id", "importer_id"])
        .agg({"value": "sum"})
    )

In [ ]:
complex_query_from_disk_pd()

#### polars

In [ ]:
def complex_query_pl_from_disk() -> pl.DataFrame:

    plan = (
        pl.scan_parquet(DATA_PATH)
        .filter(
            (pl.col("exporter_id").str.contains("A|B"))
            & (
                pl.col("product_name").str.ends_with("s")
                | pl.col("product_name").str.starts_with("s")
            )
        )
        .with_columns((pl.col("value") / pl.col("quantity")).alias("price_per_unit"))
        .group_by(["year", "exporter_id", "importer_id"])
        .agg(pl.col("value").sum())
    )
    return plan.collect()

In [ ]:
complex_query_pl_from_disk()